# WebScraping with Selenium

This notebook shows how use selenium to scrape data from Kickstarter.com
The scope is only to understand the capabilities of web scraping and prepare a dataset for academic purporse.

Let’s begin writing our scraper!

We will first install important modules and packages for our Notebook


*   Selenium
*   Chromium-chromedriver

<a href="https://colab.research.google.com/drive/1_WyM24eXWf-pdcqJKCcsb1pWwnbEWRi8?authuser=2#scrollTo=VYfg3I_fSt6e"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

And now we will import some modules on our Notebook


1.   sys: to setup the path of chrome driver
2.   selenium: to emulate the user behaviour
3.   pandas: to work with data
4.   tqdm: to show a progress bar in our notebook
5.   json: to work with json format





In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint

First we need to set the options for the ghost browser.
The most important is `--headless` because we are in a "cloud" ntotebook. In our local notebook we can remove the `--headless` option.  


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

`webdriver` is the most importa object in Selenium.
With `webdriver` we can start the phantom browser, emulate the user navigation and scrape our data. 

Let’s now create a new instance of google chrome.


We will navitage to page by a `get` request. With http we can do a get or a post (or others but they are not important for us): https://www.w3schools.com/tags/ref_httpmethods.asp 



```
wd.get("https://www.kickstarter.com/discover/advanced?sort=magic&ref=nav_search&page=1")
```



In [ ]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.kickstarter.com/discover/advanced?sort=magic&ref=nav_search&page=1")

Take a look to the screenshot from our phantom browser.

Note:
- la language and the locale: It is en_US!
- the screen ration: is it the same of our notebook?

In [ ]:
wd.save_screenshot('screenshot.png')

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/screenshot.png')
imgplot = plt.imshow(img)
plt.show()

Let's try to extract some information from the page



`title` contains the title of our page.




In [ ]:
print(wd.title)

### First exercise with *Selenium*

Try it yourself... Print the contents of the page on the screen!

Complete the code and annotate the different behaviour. Try to use the attributes/methods applied to `wd`

- `page_source`
- `find_element_by_tag_name("body")`
- `find_element_by_tag_name("body").text`




In [ ]:
print(....)

### How do we extract the values inside a page with CSS Selector?

Selenium has a method called `find_elements_by_css_selector`.

We will pass our CSS Selector into this method and get a list of selenium elements. Once we have the element, we can extract the text inside it using the `text` function.

In [ ]:
list_h3 = wd.find_elements_by_css_selector("h3")
print(len(list_h3))

In [ ]:
print(list_h3[0].text)

### Exercise on *find_elements_by_css_selector*

Try to export the list of `p` elements and show the content of the first element.




In [ ]:
list_p = wd......("...")
print(len(list_p))

# Project List from Kickstarter

Let's begin to download the list of projects.

The list of projects is defined by the css selection rule
`#projects_list > div > div.js-react-proj-card`



In [ ]:
list_projects = wd.find_elements_by_css_selector("#projects_list > div > div.js-react-proj-card")
print(len(list_projects))

Gread! We find 12 project!

Now, we will extract the attribute from each single element.
The scope is to create one dict for each project in the list with the attribute:


1.   url
2.   title
3.   description
4.   category
5.   location



In [ ]:
import pprint
detail_projects = []
for project in list_projects:
    title = project.find_elements_by_css_selector("h3")[0].text
    url = project.find_elements_by_css_selector("a")[0].get_attribute("href")
    description = project.find_elements_by_css_selector(".mb3 p")[0].text
    category = project.find_elements_by_css_selector(".pb3 a")[4].text
    location = project.find_elements_by_css_selector(".pb3 a")[5].text
    project_id = project.get_attribute("data-pid")

    detail_projects.append({'url': url,
                            'title': title,
                            'description': description,
                            'category': category,
                            'location': location,
                            'project_id': project_id})

len(detail_projects)
pprint.pprint(detail_projects)

Now let's try downloading all the ads on the front page...
and then move on to the second.

First we define a `parse_project` function to make the code cleaner and to reuse the function.

In [ ]:
def parse_project(project):
  project_id = project.get_attribute("data-pid")
  title = ""
  description = ""
  category = ""
  location = ""
  try:
    title = project.find_elements_by_css_selector("h3")[0].text
    url = project.find_elements_by_css_selector("a")[0].get_attribute("href")
    description = project.find_elements_by_css_selector(".mb3 p")[0].text
    category = project.find_elements_by_css_selector(".pb3 a")[4].text
    location = project.find_elements_by_css_selector(".pb3 a")[5].text
  except:
    pass
  return {'url': url,
          'title': title,
          'description': description,
          'category': category,
          'location': location,
          'project_id': project_id}


Here's how to download the first two pages of projects...

In [ ]:
detail_projects = []
for num in tqdm(range(1,3)):
  wd.get(f"https://www.kickstarter.com/discover/advanced?sort=magic&ref=nav_search&page={num}")
  wd.save_screenshot(f'screenshot_{num}.png')
  list_projects = wd.find_elements_by_css_selector("#projects_list > div > div.js-react-proj-card")
  for project in list_projects:
    detail_projects.append(parse_project(project))

print(len(detail_projects))

Have you seen the bookstore **tqdm**!
Find at this link all the documentation **https://github.com/tqdm/tqdm**:
is very useful to make our notebook more nice...

### How to end scraping?
Problem: When do we stop?

There are several ways. Some simpler and some more advanced:
- We can search inside the page for the ad number and divide it by 12
- We can check the status of the request (if (status_code == 200):... for a list of possible http status codes please check https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- We can check the number of ads found (if > 0...)
- Other more advanced techniques...

For example, in our case, let's go to verify the presence, or not, of the *Load more* button inside the page. If present, we continue scraping, otherwise we stop.

Actually, we can also combine more than one of these techniques.

For this exercise we will just download the first 10 pages.

In [ ]:
import time
detail_projects = []
for num in tqdm(range(1,100)):
  time.sleep(1)
  wd.get(f"https://www.kickstarter.com/discover/advanced?sort=magic&ref=nav_search&page={num}")
  wd.save_screenshot(f'screenshot_{num}.png')
  list_projects = wd.find_elements_by_css_selector("#projects_list > div > div.js-react-proj-card")
  for project in list_projects:
    detail_projects.append(parse_project(project))

print(len(detail_projects))

# Pandas and data processing

**Well!**
We're starting to see another amazing library for working with data!

It's **pandas**: *Python library for data analysis*.

Basically, with pandas we can manipulate a data set or a historical series in Python.

For now, we start to give a couple of concepts.

Pandas is based on two types of data: **Series** and **DataFrame**:
- `Series` represents a list of data
- `DataFrame` represent a data set in tabular format

Each column of a `DataFrame` is a `Series`.

We can create a `DataFrame` using the `pd.DataFrame` method by passing our dictionary as an input parameter.

A `DataFrame` pandas can be easily exported in *CSV*, *Excel*, ...

For more information about `Pandas` see

https://pandas.pydata.org/getting_started.html


In [ ]:
import pandas as pd
ds_detail_projects = pd.DataFrame(detail_projects)
ds_detail_projects.set_index("project_id")
ds_detail_projects.head()

The `.info()` method provides an indication of the structure and data of the `DataFrame`.

In [ ]:
ds_detail_projects.info()

In [ ]:
ds_detail_projects.to_csv('ds_projects.csv')

# Kickstarter Project pages ###

Now, the goal is to navigate and download the details of each project and pictures of the houses.

**Pandas** provides the *.read_csv* method that allows you to upload in CSV format files within a DataFrame.

In [ ]:
# open csv file
import pandas as pd
ds_detail_projects = pd.read_csv("ds_projects.csv", index_col="project_id")
ds_detail_projects.head()

***For each project we go to the link... how do you think we do it?***

Let's try the first 5 projects.



These days most of the web apps are using AJAX techniques. When a page is loaded by the browser, the elements within that page may load at different time intervals. Using waits, we can stop our scraping and wait the loading of a element.

Selenium Webdriver provides two types of waits - implicit & explicit. An explicit wait makes WebDriver wait for a certain condition to occur before proceeding further with execution. An implicit wait makes WebDriver poll the DOM for a certain amount of time when trying to locate an element.


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
details = []
for project_id, project in ds_detail_projects.head().iterrows():
    time.sleep(10)
    link = project["url"]
    print(link)
    wd.set_window_size(1920, 1080)
    wd.get(link)
    try:
      WebDriverWait(wd, 10).until(
          EC.element_to_be_clickable((By.ID, "back-project-button"))
      )
    except:
      continue

    wd.save_screenshot(f'screenshot_{project_id}.png')
    backers = wd.find_elements_by_css_selector(".type-16.dark-grey-500")[0].text
    goal = wd.find_elements_by_css_selector(".ksr-green-500")[0].text
    pledged = wd.find_elements_by_css_selector("span.money")[1].text
    details.append({'project_id': project_id,
                    'backers': backers,
                    'goal': goal,
                    'pledged': pledged})
  
len(details)
pprint.pprint(details)


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
details = []
for project_id, project in ds_detail_projects.iterrows():
    time.sleep(2)
    link = project["url"]
    backers = 0
    goal = 0
    pledged = 0
    #print(link)
    try:
      wd.set_window_size(1920, 1080)
      wd.get(link)
      try:
        WebDriverWait(wd, 10).until(
            EC.element_to_be_clickable((By.ID, "back-project-button"))
        )
      except:
        continue

      wd.save_screenshot(f'screenshot_{project_id}.png')
      backers_list = wd.find_elements_by_css_selector(".type-16.dark-grey-500")
      goal_list = wd.find_elements_by_css_selector(".ksr-green-500")
      pledged_list = wd.find_elements_by_css_selector("span.money")
      if(len(backers_list) > 0):
        backers = int(backers_list[0].text.replace(",", ""))
      if(len(goal_list) > 0):
        goal = int(goal_list[0].text.replace(",", "").replace("$", "").replace("US", ""))
      if(len(pledged_list) > 1):
        pledged = int(pledged_list[1].text.replace(",", "").replace("$", "").replace("US", ""))
    except Exception as e:
      print(e)
    details.append({'project_id': project_id,
                    'backers': backers,
                    'goal': goal,
                    'pledged': pledged})
  
print(len(details))
#pprint.pprint(details)


In [ ]:
len(details)

Store the data with *pandas*

In [ ]:
import pandas as pd
ds_details = pd.DataFrame(details)
ds_details.set_index("project_id")
ds_details.head()

In [ ]:
ds_details.info()

In [ ]:
ds_details.to_csv("ds_project_details.csv")

# Kickstarter Project images


Now we're going to download Kickstarter images for each project.

Our goal is to create a dataset of images and a `Dataframe` composed by:
- `project_id`
- `image_id`

We will use the library `requests`.

The `requests` library is python library for making HTTP requests.
It abstracts the complexities of making requests behind a simple API so that you can focus on interacting with services and consuming data in your application.

In [ ]:
import time
import requests

detail_projects = []
wd.get("https://www.kickstarter.com/discover/advanced?ref=nav_search&term=")
for num in tqdm(range(1,100)):
  time.sleep(10)
  list_projects = wd.find_elements_by_css_selector("#projects_list > div > div.js-react-proj-card")
  list_images = []
  for project in list_projects:
    try:
      project_id = project.get_attribute("data-pid")
      src = project.find_element_by_css_selector("a img").get_attribute("src")
      img_file = requests.get(src, stream=True)
      if img_file.status_code == 200:
        with open("/content/images/img_" + str(project_id) + ".jpg", 'wb') as f:
          f.write(img_file.content)
        list_images.append({
            "project_id": project_id,
            "img_file": "img_" + str(project_id) + ".jpg"})
    except Exception as e:
      print(e)
  load_more = wd.find_elements_by_css_selector(".load_more")
  if(len(load_more)) > 0:
    load_more[0].click()
  else:
    break

print(len(list_images))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/images/img_142016465.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
import pandas as pd
ds_images = pd.DataFrame(list_images)
ds_images.set_index("project_id")
ds_images.head()

In [ ]:
ds_images.to_csv("ds_images.csv")

In [ ]:
!zip -r "/content/images.zip" "/content/images/"


# API

Let's see how to use the **requests** library to hook APIs provided by our suppliers or colleagues.

## Crarifai

Let's try the services provided by Clarifai to increase the data we have available.


First, we need to install the `crarifai` python library.
For more details, please visit https://github.com/Clarifai/clarifai-python

To install the API client:


```
pip install clarifai
```






In [ ]:
!pip install clarifai

In [ ]:
from clarifai.rest import ClarifaiApp

# setup your key!!!
clarifai_key = "<your_key>"
app = ClarifaiApp(api_key=clarifai_key)

# and use the general model
model = app.public_models.general_model

We will use the `predict_by_filename` method to obtain the **concepts** within the page.


In [ ]:
response = model.predict_by_filename("/content/images/img_142016465.jpg")

What type of data does that give us back?
Let's have a look!

In [ ]:
pprint.pprint(response)

And get the data:

In [ ]:
if(response['status']['description'] == "Ok"):
    for concept in response["outputs"][0]["data"]["concepts"]:
        name = concept["name"]
        value = concept["value"]
        print(name + " " + str(value))

Now let's go and identify all the concepts for each image of each project.

In [ ]:
ds_images = pd.read_csv("ds_images.csv", index_col="project_id")

img_details = []
for project_id, image in ds_images.iterrows():
  try:
    response = model.predict_by_filename("/content/images/" + image['img_file'])
    if(response['status']['description'] == "Ok"):
      for concept in response["outputs"][0]["data"]["concepts"]:
          name = concept["name"]
          value = concept["value"]
          img_details.append({
            "project_id": project_id,
            "image": image['img_file'],
            "name": name,
            "value": value
          })
  except Exception as e:
    print(e)

print(len(img_details))

In [ ]:
import pandas as pd
ds_img_details = pd.DataFrame(img_details)
ds_img_details.set_index("image")
ds_img_details.head()

In [ ]:
ds_img_details.to_csv('ds_img_details.csv')

# Geocoding

What's geocoding?



Let's try to geolocalize the projects...

In [ ]:
address = "via finazzi 46, carvico(bg)"
key = "<your_key>"
geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={address}"
response = requests.get(geocode_url)

import json
geo = json.loads(response.text)
pprint.pprint(response.text)

In [ ]:
print(geo['results'][0]['locations'][0]['latLng']['lat'])
print(geo['results'][0]['locations'][0]['latLng']['lng'])

In [ ]:
ds_projects = pd.read_csv("ds_projects.csv", index_col="project_id")

project_location = []
for project_id, project in ds_projects.iterrows():
  address = project['location']
  key = "<your_key>"
  try:
    geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={address}"
    response = requests.get(geocode_url)
    if(response.status_code == 200):
      geo = json.loads(response.text)
      lat = geo['results'][0]['locations'][0]['latLng']['lat']
      lng = geo['results'][0]['locations'][0]['latLng']['lng']
      project_location.append({
            "project_id": project_id,
            "lat": lat,
            "lng": lng
          })
  except Exception as e:
    print(e)
    

print(len(project_location))

In [ ]:
import pandas as pd
ds_project_location = pd.DataFrame(project_location)
ds_project_location.set_index("project_id")
ds_project_location.head()

In [ ]:
ds_project_location.to_csv('ds_project_location.csv')